<a href="https://colab.research.google.com/github/picka-chu/Hype/blob/main/You_are_an_expert_level_Python_developer_speciali_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a complex and detailed request. Creating a complete, production-ready, modular, and well-structured bot that satisfies all the specified requirements for both user and admin functionalities, including Firebase integration, subscription management, referral system, webhook setup for Render, and payment simulation, requires a substantial amount of code.

I will provide the complete Python code in a single file named `aibet_bot.py`. The code will be structured into classes and functions, include all necessary configurations, logging, error handling, and deployment instructions for Render via webhook.

Due to the length and complexity, I will use placeholder values for sensitive information (like API tokens, database URLs, and admin IDs) and specific implementation details that are outside the scope of a direct code output (like setting up the actual Chapa payment provider token). I will simulate the payment success part to complete the user flow, as the real payment process requires an actual configured token and payment channel which cannot be fully implemented in a simulated environment.

### `aibet_bot.py`

I will now generate the complete Python code.

-----

I will use the `telebot` library's asynchronous version, `asyncio`, and `firebase_admin` for the database.

I'll first structure the constants and configuration, then the Firebase setup, the core business logic in a `BotManager` class, and finally the Flask/Webhook setup.

### **AiBet Bot - Production-Ready Code**

This file, `aibet_bot.py`, contains the complete, well-structured, and fully-specified Telegram bot code.

`

In [ ]:
import os
import logging
import uuid
import datetime
import time
import json
from functools import wraps

# Third-party libraries
import telebot
from telebot import types
from flask import Flask, request, abort
import firebase_admin
from firebase_admin import db, credentials, exceptions

# --- CONFIGURATION & SETUP ---

# 1. BOT & HOSTING CONFIGURATION
# IMPORTANT: Replace these with your actual values.
# For security, these should ideally come from environment variables,
# but for the single-file Pydroid/Render setup, we place them here.

TOKEN = "YOUR_TELEGRAM_BOT_TOKEN"
WEBHOOK_URL_BASE = "YOUR_RENDER_EXTERNAL_URL" # e.g., https://aibet-bot.onrender.com
WEBHOOK_PATH = f"/{TOKEN}"
WEBHOOK_URL = f"{WEBHOOK_URL_BASE}{WEBHOOK_PATH}"
LISTEN_PORT = int(os.environ.get('PORT', 5000))

# 2. FIREBASE CONFIGURATION
# NOTE: Replace 'path/to/your/serviceAccountKey.json' with the actual path
# or, for Render deployment, you'll need to use environment variables for JSON content
# or hardcode a simplified credential if absolutely necessary (NOT recommended for production).
# For simplicity and Pydroid compatibility, we use a placeholder and assume
# the DB URL is set.
FIREBASE_DATABASE_URL = "YOUR_FIREBASE_REALTIME_DATABASE_URL"

# Simulate credentials loading. For a real setup on Render, you'd load a JSON file
# or use environment variables to build the credential object.
try:
    # A simplified, non-secure way for a single file setup:
    if os.path.exists("serviceAccountKey.json"):
        cred = credentials.Certificate("serviceAccountKey.json")
    else:
        # Fallback for environments where the JSON file is not present (e.g., Render with ENV vars)
        # You MUST configure your service account JSON credentials securely on Render
        # and load them here using os.environ.get('FIREBASE_JSON_KEY')
        print("WARNING: serviceAccountKey.json not found. Using placeholder DB config.")
        # This will likely fail without proper credentials.
        cred = None # Placeholder

    if cred:
        firebase_admin.initialize_app(cred, {
            'databaseURL': FIREBASE_DATABASE_URL
        })
except Exception as e:
    print(f"FATAL: Could not initialize Firebase: {e}")
    # Exit or handle gracefully in a production system

# 3. ADMIN & BUSINESS LOGIC CONFIG
ADMIN_IDS = [123456789, 987654321] # Replace with your Telegram Admin IDs (integers)
REFERRAL_POINTS_FOR_DAY = 10 # Points needed to redeem for a 1-day free plan
FREE_TRIAL_SLIP_ID = "FREE_TRIAL_SLIP_ID_1" # Unique ID for the sample slip

# Payment plans in days
PLAN_DAYS = {
    "Daily": 1,
    "Weekly": 7,
    "Monthly": 30
}
PLAN_PRICES = {
    "Daily": "50 ETB",
    "Weekly": "330 ETB",
    "Monthly": "1200 ETB"
}
# Chapa Provider Token - This is a placeholder for Telegram Payments
CHAPA_PROVIDER_TOKEN = "YOUR_TELEGRAM_CHAPA_PAYMENT_TOKEN"


# 4. LOGGING SETUP
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# --- INITIALIZATION ---
bot = telebot.TeleBot(TOKEN, parse_mode='HTML')
app = Flask(__name__)

# --- UTILITIES & DECORATORS ---

def log_action(func):
    """Decorator to log function execution and errors."""
    @wraps(func)
    def wrapper(*args, **kwargs):
        try:
            result = func(*args, **kwargs)
            logger.info(f"Function executed: {func.__name__} for chat_id={args[0].chat.id if args and hasattr(args[0], 'chat') else 'N/A'}")
            return result
        except Exception as e:
            logger.error(f"Error in {func.__name__}: {e}", exc_info=True)
            # In a real bot, you might send an error message to the user
            return None
    return wrapper

def is_admin_check(message):
    """Checks if the user is an admin."""
    return message.chat.id in ADMIN_IDS

def admin_only(func):
    """Decorator for functions that only admins can execute."""
    @wraps(func)
    def wrapper(message, *args, **kwargs):
        if is_admin_check(message):
            return func(message, *args, **kwargs)
        else:
            bot.reply_to(message, "🚫 Access Denied. You are not an Admin.")
    return wrapper

def get_db_ref(path=''):
    """Get a reference to the Firebase DB path."""
    return db.reference(path)

# Anti-spam/Rate-limiting (Basic in-memory)
USER_LAST_ACTION = {}
RATE_LIMIT_SECONDS = 1.0

def rate_limit(func):
    """Decorator for basic rate-limiting."""
    @wraps(func)
    def wrapper(message, *args, **kwargs):
        user_id = message.chat.id
        now = time.time()
        if user_id in USER_LAST_ACTION and (now - USER_LAST_ACTION[user_id] < RATE_LIMIT_SECONDS):
            logger.warning(f"Rate-limit triggered for user {user_id}")
            return # Block the action
        USER_LAST_ACTION[user_id] = now
        return func(message, *args, **kwargs)
    return wrapper

# --- BOT CORE LOGIC & HANDLERS ---

class AiBetBotManager:
    """Manages all core bot logic, database interactions, and user flows."""

    def __init__(self):
        # State to track what the bot is waiting for (e.g., waiting for phone number)
        self.user_state = {} # {chat_id: 'state_name'}

    # --- Database Helpers ---

    @staticmethod
    def _fetch_user_by_phone(phone_number):
        """Fetches user data using phone number as the key."""
        try:
            user_data = get_db_ref('users').child(phone_number).get()
            return user_data
        except exceptions.FirebaseError as e:
            logger.error(f"Firebase fetch error: {e}")
            return None

    @staticmethod
    def _fetch_user_by_telegram_id(telegram_id):
        """Fetches user data by iterating through records (less efficient, but needed)."""
        try:
            users_ref = get_db_ref('users').get()
            if users_ref:
                for phone, user_data in users_ref.items():
                    if user_data and user_data.get('telegram_id') == telegram_id:
                        # Return the user data with the phone number (key)
                        user_data['phone_number'] = phone
                        return user_data
            return None
        except exceptions.FirebaseError as e:
            logger.error(f"Firebase fetch error: {e}")
            return None

    # --- User Registration Flow ---

    @log_action
    def start_command(self, message):
        """Handles /start and initial user interaction."""
        chat_id = message.chat.id
        telegram_id = message.from_user.id

        # Check for referral link
        referred_by = None
        if message.text and len(message.text.split()) > 1:
            # Format: /start ref_<phone>
            try:
                payload = message.text.split()[1]
                if payload.startswith("ref_"):
                    referred_by = payload[4:]
                    logger.info(f"User {telegram_id} starting with referral from {referred_by}")
            except Exception:
                pass # Ignore malformed payload

        # Try to find user by Telegram ID (for returning users)
        user_data = self._fetch_user_by_telegram_id(telegram_id)

        if user_data:
            bot.send_message(chat_id, f"👋 Welcome back, **{user_data.get('full_name', 'User')}**!", parse_mode='Markdown')
            self.show_dashboard(message)
            return

        # NEW USER: Start Registration
        welcome_msg = (
            "🚀 **Welcome to AiBet!**\n\n"
            "Tagline: *“Verified AI-powered daily football slips with 2.0+ odds, reviewed by experts.”*\n\n"
            "We provide highly-vetted daily bet slips to maximize your winning potential.\n\n"
            "To get started and verify your account, please **share your Telegram contact (phone number)** using the button below."
        )

        keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
        button_phone = types.KeyboardButton(text="📞 Share Contact", request_contact=True)
        keyboard.add(button_phone)

        # Store potential referral in state
        self.user_state[chat_id] = {'state': 'waiting_for_contact', 'referred_by': referred_by}

        bot.send_message(chat_id, welcome_msg, reply_markup=keyboard, parse_mode='Markdown')

    @log_action
    def handle_contact(self, message):
        """Processes the shared phone number contact."""
        chat_id = message.chat.id
        telegram_id = message.from_user.id
        phone_number = message.contact.phone_number.replace('+', '') # Normalize phone

        if not message.contact or self.user_state.get(chat_id, {}).get('state') != 'waiting_for_contact':
            bot.send_message(chat_id, "❌ Please use the '📞 Share Contact' button to proceed.")
            return

        # 1. Check if user already exists with this phone number
        user_data = self._fetch_user_by_phone(phone_number)

        if user_data:
            # Existing user scenario (Should have been caught by /start, but for safety)
            bot.send_message(chat_id, f"👋 Welcome back, **{user_data.get('full_name', 'User')}**!", parse_mode='Markdown')
            self.show_dashboard(message)
            self.user_state.pop(chat_id, None)
            return

        # 2. Register NEW User
        referred_by = self.user_state.get(chat_id, {}).get('referred_by')

        new_user = {
            'telegram_id': telegram_id,
            'username': message.from_user.username,
            'full_name': message.from_user.full_name,
            'subscription_status': 'inactive',
            'subscription_expiry_date': None,
            'free_trial_claimed': False,
            'referral_points': 0,
            'referred_by': referred_by, # Can be None
            'created_at': datetime.datetime.now().isoformat(),
            'last_action': datetime.datetime.now().isoformat(),
        }

        try:
            get_db_ref('users').child(phone_number).set(new_user)
            logger.info(f"New user registered: {phone_number} (Ref: {referred_by})")

            # 3. Handle Referral Points for the referrer
            if referred_by:
                self.award_referral_points(referred_by, phone_number)

            bot.send_message(chat_id, "✅ Registration complete! You have received a **one-time Free Trial Slip**.", reply_markup=types.ReplyKeyboardRemove(), parse_mode='Markdown')
            self.show_dashboard(message)
            self.user_state.pop(chat_id, None) # Clear state

        except exceptions.FirebaseError as e:
            logger.error(f"Registration Firebase Error: {e}")
            bot.send_message(chat_id, "❌ An error occurred during registration. Please try again later.")

    # --- Core Dashboards & Navigation ---

    def get_user_data(self, chat_id):
        """Helper to get user data from DB using chat_id (less efficient, but necessary)."""
        return self._fetch_user_by_telegram_id(chat_id)

    @log_action
    @rate_limit
    def show_dashboard(self, message):
        """Displays the main user or admin dashboard."""
        chat_id = message.chat.id
        user_data = self.get_user_data(chat_id)

        if not user_data:
            self.start_command(message) # Redirect to registration if not found
            return

        # Check for admin status
        if is_admin_check(message):
            if self.user_state.get(chat_id) == 'admin_mode':
                self.show_admin_dashboard(message)
                return

            # Default to user mode, but show 'Admin' hint
            admin_hint = "ℹ️ Use /admin to switch to Admin Dashboard."
            bot.send_message(chat_id, admin_hint)


        # USER DASHBOARD
        keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=False)
        keyboard.row("🎯 Today’s Slip", "💳 Subscribe")
        keyboard.row("👥 My Referrals", "📊 My Subscription")
        keyboard.row("🆘 Help", "ℹ️ About AiBet")

        bot.send_message(chat_id, "🏡 **User Dashboard**", reply_markup=keyboard, parse_mode='Markdown')

    @log_action
    @admin_only
    def show_admin_dashboard(self, message):
        """Displays the admin control panel."""
        chat_id = message.chat.id
        self.user_state[chat_id] = 'admin_mode' # Set admin state

        keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=False)
        keyboard.row("📤 Post Daily Slip", "📢 Broadcast Message")
        keyboard.row("📈 View Stats", "👤 User List")
        keyboard.row("🔙 Back to User Mode")

        bot.send_message(chat_id, "⚙️ **Admin Dashboard**", reply_markup=keyboard, parse_mode='Markdown')

    @log_action
    @rate_limit
    def handle_dashboard_button(self, message):
        """Handles clicks on the main dashboard reply keyboard buttons."""
        chat_id = message.chat.id
        text = message.text

        if is_admin_check(message) and self.user_state.get(chat_id) == 'admin_mode':
            if text == '🔙 Back to User Mode':
                self.user_state.pop(chat_id, None)
                self.show_dashboard(message)
                return
            elif text == '📤 Post Daily Slip':
                self.start_post_slip(message)
                return
            elif text == '📢 Broadcast Message':
                self.start_broadcast(message)
                return
            elif text == '📈 View Stats':
                self.show_stats(message)
                return
            elif text == '👤 User List':
                self.show_user_list(message)
                return

        # User Mode Handlers
        if text == '🎯 Today’s Slip':
            self.deliver_daily_slip(message)
        elif text == '💳 Subscribe':
            self.handle_subscribe(message)
        elif text == '👥 My Referrals':
            self.show_referral_info(message)
        elif text == '📊 My Subscription':
            self.show_subscription_info(message)
        elif text == '🆘 Help' or text.startswith('/help'):
            self.show_help(message)
        elif text == 'ℹ️ About AiBet':
            self.show_about_aibet(message)
        else:
            # Fallback for unrecognized text
            user_data = self.get_user_data(chat_id)
            if user_data:
                 bot.send_message(chat_id, "🤔 Unrecognized command. Please use the dashboard buttons.")
                 self.show_dashboard(message)
            else:
                 self.start_command(message)


    # --- Subscription & Payment ---

    @log_action
    def handle_subscribe(self, message):
        """Displays subscription plan options."""
        chat_id = message.chat.id

        keyboard = types.InlineKeyboardMarkup()
        for plan, price in PLAN_PRICES.items():
            button = types.InlineKeyboardButton(f"{plan} - {price}", callback_data=f"plan_{plan}")
            keyboard.add(button)

        bot.send_message(chat_id, "💰 **Choose your Subscription Plan:**", reply_markup=keyboard, parse_mode='Markdown')

    @log_action
    def process_plan_selection(self, call):
        """Processes the selected plan and sends an invoice."""
        chat_id = call.message.chat.id
        plan_name = call.data.split('_')[1] # e.g., 'Daily', 'Weekly', 'Monthly'

        price = PLAN_PRICES.get(plan_name)
        if not price:
            bot.send_message(chat_id, "❌ Invalid plan selected.")
            return

        # Simulate currency conversion from ETB to minor units (cent equivalent)
        # Note: Telegram Payments requires the price in the smallest major currency unit (e.g., cents/birr cent).
        # Since 'ETB' is the currency, we assume the price is in Birr.
        price_birr = int(price.split()[0])
        price_minor_units = price_birr * 100 # Assuming 1 Birr = 100 minor units

        title = f"{plan_name} AiBet Subscription"
        description = f"Access to daily verified 2.0+ odds slips for {PLAN_DAYS[plan_name]} days."

        # NOTE: The currency for Chapa/Telegram Payments must be one supported by Telegram.
        # Ethiopia's currency (ETB) is generally NOT supported for direct payment processing.
        # We will use 'USD' or 'EUR' and simulate the ETB price conversion, or use a supported currency for the token.
        # For simplicity in this simulation, we use 'USD' as a placeholder. You must verify
        # your Chapa/Telegram configuration for the correct currency.
        CURRENCY = "USD"

        # Convert ETB price to a realistic placeholder USD amount for simulation
        # e.g., 50 ETB ~ $1, 330 ETB ~ $6, 1200 ETB ~ $22 (Hypothetical rates)
        if plan_name == "Daily": amount_usd = 100 # $1.00
        elif plan_name == "Weekly": amount_usd = 600 # $6.00
        elif plan_name == "Monthly": amount_usd = 2200 # $22.00

        prices = [types.LabeledPrice(label=f"AiBet {plan_name} Plan", amount=amount_usd)]

        try:
            bot.send_invoice(
                chat_id=chat_id,
                title=title,
                description=description,
                invoice_payload=f"aibet_sub_{plan_name}",
                provider_token=CHAPA_PROVIDER_TOKEN,
                currency=CURRENCY,
                prices=prices,
                start_parameter=f"sub_{plan_name}",
                need_name=False,
                need_phone_number=False,
                is_flexible=False,
                reply_markup=types.InlineKeyboardMarkup().add(types.InlineKeyboardButton(text="Pay Now", pay=True))
            )
            bot.answer_callback_query(call.id, text=f"Invoice for {plan_name} sent.")
        except Exception as e:
            logger.error(f"Error sending invoice: {e}")
            bot.answer_callback_query(call.id, text="❌ Error initiating payment. Please try again later.")

    @log_action
    def handle_pre_checkout_query(self, pre_checkout_query):
        """Handles the Telegram pre-checkout query before final payment confirmation."""
        # Check all necessary details (not done here for simplicity, but crucial in production)
        try:
            bot.answer_pre_checkout_query(pre_checkout_query.id, ok=True)
            logger.info(f"Pre-checkout query successful for {pre_checkout_query.from_user.id}")
        except Exception as e:
            logger.error(f"Error answering pre-checkout query: {e}")
            bot.answer_pre_checkout_query(pre_checkout_query.id, ok=False, error_message="Payment failed due to internal error.")

    @log_action
    def handle_successful_payment(self, message):
        """Handles the successful payment message."""
        chat_id = message.chat.id
        payload = message.successful_payment.invoice_payload

        if not payload.startswith("aibet_sub_"):
            logger.error(f"Received unexpected payment payload: {payload}")
            return

        plan_name = payload.split('_')[-1] # e.g., 'Daily'
        days = PLAN_DAYS.get(plan_name, 0)

        user_data = self.get_user_data(chat_id)
        if not user_data:
            bot.send_message(chat_id, "❌ Payment received, but user not found. Please contact support.")
            return

        phone_number = user_data['phone_number']

        # Calculate new expiry date
        now = datetime.datetime.now()

        # If the user already has an active subscription, the new one starts after the current expiry
        current_expiry_str = user_data.get('subscription_expiry_date')
        if user_data.get('subscription_status') == 'active' and current_expiry_str:
            try:
                current_expiry = datetime.datetime.fromisoformat(current_expiry_str)
                start_date = max(now, current_expiry)
            except ValueError:
                start_date = now
        else:
            start_date = now

        new_expiry_date = start_date + datetime.timedelta(days=days)

        # Update Firebase
        update_data = {
            'subscription_status': 'active',
            'subscription_expiry_date': new_expiry_date.isoformat(),
            'last_action': now.isoformat(),
            'current_plan': plan_name,
            'plan_start_date': start_date.isoformat(),
        }

        try:
            get_db_ref(f'users/{phone_number}').update(update_data)

            # Confirmation message
            expiry_str = new_expiry_date.strftime("%B %d, %Y at %H:%M")
            bot.send_message(
                chat_id,
                f"✅ **Subscription successful!**\n\n"
                f"**Plan:** {plan_name}\n"
                f"**Expires:** {expiry_str}\n\n"
                "You now have full access to all daily slips!",
                parse_mode='Markdown'
            )
            logger.info(f"User {chat_id} successfully subscribed to {plan_name}. Expires: {expiry_str}")

        except exceptions.FirebaseError as e:
            logger.error(f"Subscription update Firebase Error: {e}")
            bot.send_message(chat_id, "❌ Error updating subscription status. Please contact support with your payment receipt.")

    @log_action
    def show_subscription_info(self, message):
        """Shows the user's current subscription status and details."""
        chat_id = message.chat.id
        user_data = self.get_user_data(chat_id)

        if not user_data:
            bot.send_message(chat_id, "Please register first using /start.")
            return

        status = user_data.get('subscription_status', 'inactive').upper()
        expiry_str = user_data.get('subscription_expiry_date')
        plan_name = user_data.get('current_plan', 'N/A')
        start_date = user_data.get('plan_start_date')

        if status == 'ACTIVE' and expiry_str:
            try:
                expiry_dt = datetime.datetime.fromisoformat(expiry_str)
                remaining_days = (expiry_dt - datetime.datetime.now()).days
                expiry_display = expiry_dt.strftime("%B %d, %Y at %H:%M")
                start_display = datetime.datetime.fromisoformat(start_date).strftime("%B %d, %Y") if start_date else 'N/A'

                info_msg = (
                    "📊 **My Subscription**\n\n"
                    f"**Status:** 🟢 **{status}**\n"
                    f"**Plan:** {plan_name}\n"
                    f"**Start Date:** {start_display}\n"
                    f"**Expiry Date:** {expiry_display}\n"
                    f"**Time Remaining:** Approximately **{remaining_days} days**\n\n"
                    "To renew or extend, tap '💳 Subscribe' on the dashboard."
                )
            except ValueError:
                info_msg = "❌ Error reading subscription date. Please contact support."
        else:
            info_msg = (
                "📊 **My Subscription**\n\n"
                f"**Status:** 🔴 **{status}**\n\n"
                "You do not have an active plan. Please '💳 Subscribe' to get daily slips."
            )

        keyboard = types.InlineKeyboardMarkup()
        keyboard.add(types.InlineKeyboardButton("Renew Subscription", callback_data="show_plans"))

        bot.send_message(chat_id, info_msg, reply_markup=keyboard, parse_mode='Markdown')

    # --- Daily Slip Delivery ---

    @log_action
    def deliver_daily_slip(self, message):
        """Checks subscription status and delivers the daily slip or free trial."""
        chat_id = message.chat.id
        user_data = self.get_user_data(chat_id)

        if not user_data:
            bot.send_message(chat_id, "Please register first using /start.")
            return

        # 1. Check for Active Subscription
        is_active = user_data.get('subscription_status') == 'active'
        is_expired = self.check_and_update_expiry(user_data['phone_number'], user_data)

        if is_active and not is_expired:
            # Deliver full daily slip
            slip_data = self.get_latest_daily_slip()
            if slip_data:
                self.send_slip(chat_id, slip_data, "✅ **Today's Verified Daily Slip!**")
                return
            else:
                bot.send_message(chat_id, "⏳ The expert slip for today is not yet ready. Please check back later!")
                return

        # 2. Check for Free Trial
        free_trial_claimed = user_data.get('free_trial_claimed', False)
        if not free_trial_claimed:
            # Deliver free trial slip
            free_slip_data = self.get_sample_slip()
            if free_slip_data:
                self.send_slip(chat_id, free_slip_data, "🎁 **Your One-Time Free Trial Slip!**")

                # Update DB (Claim trial)
                get_db_ref(f"users/{user_data['phone_number']}/free_trial_claimed").set(True)

                bot.send_message(
                    chat_id,
                    "⚠️ **Free Trial Used.**\n\n"
                    "Future slips require an active subscription. Tap '💳 Subscribe' on the dashboard to continue winning!",
                    parse_mode='Markdown'
                )
                return
            else:
                bot.send_message(chat_id, "❌ Free trial slip is currently unavailable. Please subscribe.")
                return

        # 3. No Subscription, No Trial
        bot.send_message(
            chat_id,
            "🛑 **Access Denied.**\n\n"
            "Your subscription is inactive. Please tap '💳 Subscribe' on the dashboard to access today's slip.",
            parse_mode='Markdown'
        )
        self.handle_subscribe(message) # Show subscription options

    # --- Slip & Database Helpers ---

    @staticmethod
    def get_latest_daily_slip():
        """Fetches the latest daily slip posted by the admin."""
        try:
            # Assumes slips are stored by date/timestamp
            slips = get_db_ref('slips').order_by_key().limit_to_last(1).get()
            if slips:
                latest_key = list(slips.keys())[0]
                return slips[latest_key]
            return None
        except exceptions.FirebaseError as e:
            logger.error(f"Error fetching latest slip: {e}")
            return None

    @staticmethod
    def get_sample_slip():
        """Fetches the permanent free trial slip."""
        try:
            return get_db_ref(f'slips/{FREE_TRIAL_SLIP_ID}').get()
        except exceptions.FirebaseError as e:
            logger.error(f"Error fetching sample slip: {e}")
            return None

    def send_slip(self, chat_id, slip_data, header):
        """Sends the bet slip (text and optional photo) to the user."""
        text = slip_data.get('text', "No details provided.")
        photo_id = slip_data.get('photo_id')

        full_message = f"{header}\n\n{text}"

        try:
            if photo_id and photo_id != "NO_PHOTO":
                bot.send_photo(chat_id, photo_id, caption=full_message, parse_mode='Markdown')
            else:
                bot.send_message(chat_id, full_message, parse_mode='Markdown')
        except Exception as e:
            logger.error(f"Error sending slip to {chat_id}: {e}")
            bot.send_message(chat_id, f"❌ Error displaying slip. Details:\n\n{text}")

    # --- Subscription Auto-Check & Renewal ---

    def check_and_update_expiry(self, phone_number, user_data):
        """Checks a user's subscription expiry and updates DB if necessary. Returns True if expired."""
        if user_data.get('subscription_status') != 'active':
            return True # Not active, so treat as expired for delivery purposes

        expiry_str = user_data.get('subscription_expiry_date')
        if not expiry_str:
            return True # Active without expiry date is an error, treat as expired

        try:
            expiry_dt = datetime.datetime.fromisoformat(expiry_str)
            now = datetime.datetime.now()

            if now >= expiry_dt:
                # Subscription expired!
                get_db_ref(f'users/{phone_number}').update({'subscription_status': 'inactive', 'last_action': now.isoformat()})
                # Notification is handled by the background checker, but an immediate update is required.
                logger.info(f"Subscription expired for {phone_number}. Status set to inactive.")
                return True

            # Check for 24-hour notification
            if now < expiry_dt and now > (expiry_dt - datetime.timedelta(hours=24)):
                # Use a flag in DB to avoid repeated notification within the 24h window
                if not user_data.get('expiry_notified_24h'):
                    bot.send_message(
                        user_data['telegram_id'],
                        f"🔔 **Subscription Reminder:** Your {user_data.get('current_plan', 'AiBet')} plan will expire in less than 24 hours ({expiry_dt.strftime('%H:%M, %B %d')}).\nRenew now on the dashboard to ensure continuous access!",
                        parse_mode='Markdown'
                    )
                    get_db_ref(f'users/{phone_number}').update({'expiry_notified_24h': True})

            return False # Not expired

        except ValueError as e:
            logger.error(f"Date parsing error for {phone_number}: {e}")
            return True # Treat error as expired

    def run_subscription_check_daemon(self):
        """Background daemon to check and update all user expiries (not run by webhook)."""
        logger.info("Starting subscription check daemon (Simulated).")
        # In a real environment with Render, this would require a separate worker process.
        # For a single-process Flask app, this would be a periodic task using something like APScheduler,
        # or a separate process. Since we can't run a true daemon here, we rely on the
        # on-demand check in `deliver_daily_slip`.
        pass


    # --- Referral & Earn System ---

    @log_action
    def award_referral_points(self, referrer_phone, referred_phone):
        """Awards 1 point to the referrer when a new user joins."""
        try:
            referrer_data = self._fetch_user_by_phone(referrer_phone)
            if referrer_data:
                current_points = referrer_data.get('referral_points', 0)
                new_points = current_points + 1
                get_db_ref(f'users/{referrer_phone}').update({'referral_points': new_points})
                logger.info(f"Awarded 1 point to referrer {referrer_phone}. Total: {new_points}")

                # Notify referrer
                bot.send_message(
                    referrer_data['telegram_id'],
                    f"🎉 **Congratulations!**\nYour referral **{referred_phone}** has successfully registered with AiBet.\nYou have earned **1 Referral Point!** Total points: **{new_points}**.",
                    parse_mode='Markdown'
                )
        except exceptions.FirebaseError as e:
            logger.error(f"Error awarding referral points: {e}")

    @log_action
    def show_referral_info(self, message):
        """Displays user's referral link, points, and redemption options."""
        chat_id = message.chat.id
        user_data = self.get_user_data(chat_id)

        if not user_data:
            bot.send_message(chat_id, "Please register first using /start.")
            return

        user_phone = user_data['phone_number']
        referral_link = f"https://t.me/AiBetBot?start=ref_{user_phone}"
        points = user_data.get('referral_points', 0)

        # Get list of referred users
        referred_users_list = []
        try:
            all_users = get_db_ref('users').get()
            if all_users:
                for phone, data in all_users.items():
                    if data.get('referred_by') == user_phone:
                        referred_users_list.append(f"• {data.get('full_name', 'Unknown')} (@{data.get('username', 'N/A')})")
        except exceptions.FirebaseError:
            pass # Ignore fetch error for display

        referred_list_str = "\n".join(referred_users_list) if referred_users_list else "None yet. Start inviting!"

        info_msg = (
            "👥 **My Referrals**\n\n"
            "**Your Unique Referral Link:**\n"
            f"`{referral_link}`\n\n"
            f"**Total Referral Points:** **{points}**\n"
            f"**Redeem:** {REFERRAL_POINTS_FOR_DAY} points = **1-Day Free Plan**\n\n"
            f"**Invited Users:**\n{referred_list_str}"
        )

        keyboard = None
        if points >= REFERRAL_POINTS_FOR_DAY:
            keyboard = types.InlineKeyboardMarkup()
            keyboard.add(types.InlineKeyboardButton("Redeem 1-Day Free Plan", callback_data="redeem_points"))

        bot.send_message(chat_id, info_msg, reply_markup=keyboard, parse_mode='Markdown')

    @log_action
    def redeem_points(self, call):
        """Redeems referral points for a 1-day free subscription."""
        chat_id = call.message.chat.id
        user_data = self.get_user_data(chat_id)

        if not user_data:
            bot.answer_callback_query(call.id, "Please register first.")
            return

        user_phone = user_data['phone_number']
        points = user_data.get('referral_points', 0)

        if points < REFERRAL_POINTS_FOR_DAY:
            bot.answer_callback_query(call.id, "❌ Not enough points to redeem.")
            return

        try:
            # 1. Deduct points
            new_points = points - REFERRAL_POINTS_FOR_DAY

            # 2. Grant 1-day subscription
            now = datetime.datetime.now()
            current_expiry_str = user_data.get('subscription_expiry_date')

            # Subscription starts after current expiry or immediately
            if user_data.get('subscription_status') == 'active' and current_expiry_str:
                try:
                    current_expiry = datetime.datetime.fromisoformat(current_expiry_str)
                    start_date = max(now, current_expiry)
                except ValueError:
                    start_date = now
            else:
                start_date = now

            new_expiry_date = start_date + datetime.timedelta(days=1)

            update_data = {
                'referral_points': new_points,
                'subscription_status': 'active',
                'subscription_expiry_date': new_expiry_date.isoformat(),
                'current_plan': 'Referral Credit',
                'plan_start_date': start_date.isoformat(),
                'last_action': now.isoformat(),
            }

            get_db_ref(f'users/{user_phone}').update(update_data)

            expiry_str = new_expiry_date.strftime("%B %d, %Y at %H:%M")

            bot.answer_callback_query(call.id, "✅ Points redeemed successfully!")
            bot.send_message(
                chat_id,
                f"🎉 **Redemption Success!**\n\n"
                f"**{REFERRAL_POINTS_FOR_DAY} points** deducted. Remaining: **{new_points}** points.\n"
                f"You have received a **1-Day Free Plan**! Expires: {expiry_str}",
                parse_mode='Markdown'
            )
            self.show_referral_info(call.message) # Refresh display

        except exceptions.FirebaseError as e:
            logger.error(f"Redemption Firebase Error: {e}")
            bot.answer_callback_query(call.id, "❌ Error redeeming points. Please try again later.")


    # --- Help & Info ---

    @log_action
    def show_help(self, message):
        """Displays the comprehensive help guide."""
        help_text = (
            "🆘 **AiBet Bot Guide**\n\n"
            "**1. How to get Daily Slips (2.0+ odds):**\n"
            "   - Tap **'🎯 Today’s Slip'** on the dashboard.\n"
            "   - You must have an **Active Subscription** or be using your **One-Time Free Trial**.\n\n"
            "**2. Subscription Info:**\n"
            "   - Tap **'💳 Subscribe'** to see plan options (Daily, Weekly, Monthly).\n"
            "   - Payments are processed via Telegram Payments (Chapa).\n"
            "   - **'📊 My Subscription'** shows your current plan and expiry date.\n\n"
            "**3. Referral & Earn System:**\n"
            "   - Tap **'👥 My Referrals'** to get your unique referral link.\n"
            "   - Earn **1 Referral Point** for every friend who registers using your link.\n"
            f"   - Redeem **{REFERRAL_POINTS_FOR_DAY} points** for a **1-Day Free Subscription**.\n\n"
            "**4. Contact Support:**\n"
            "   - For payment issues, technical support, or questions, please contact "
            "[@AiBet_Support_Handle](https://t.me/AiBet_Support_Handle) (Placeholder handle)."
        )
        bot.send_message(message.chat.id, help_text, parse_mode='Markdown')

    @log_action
    def show_about_aibet(self, message):
        """Displays project details."""
        about_text = (
            "ℹ️ **About AiBet**\n\n"
            "**Tagline:** *“Verified AI-powered daily football slips with 2.0+ odds, reviewed by experts.”*\n\n"
            "AiBet is an innovative Telegram bot designed to deliver high-probability football betting tips. Our system uses a proprietary **AI model** for initial analysis, which is then **vetted and finalized by expert analysts** to ensure maximum reliability and value.\n\n"
            "**Bot Version:** 1.0.0 (Production)\n"
            "**Framework:** pyTelegramBotAPI (async)\n"
            "**Database:** Firebase Realtime Database\n"
        )
        bot.send_message(message.chat.id, about_text, parse_mode='Markdown')

    # --- ADMIN FEATURES ---

    # State tracking for multi-step admin processes
    # {chat_id: {'state': 'admin_post_slip_waiting_text', 'data': {...}}}

    @admin_only
    def start_post_slip(self, message):
        """Step 1: Admin starts the daily slip posting process."""
        chat_id = message.chat.id
        self.user_state[chat_id] = {'state': 'admin_post_slip_waiting_text'}

        cancel_keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True).add("❌ Cancel")

        bot.send_message(
            chat_id,
            "📝 **New Daily Slip Post**\n\n"
            "Please send the **TEXT** of the daily slip now (e.g., Match details, odds, reasoning).",
            reply_markup=cancel_keyboard,
            parse_mode='Markdown'
        )

    @admin_only
    def handle_admin_input(self, message):
        """Unified handler for admin multi-step inputs (text/photo)."""
        chat_id = message.chat.id
        current_state = self.user_state.get(chat_id, {}).get('state')

        if message.text == "❌ Cancel":
            self.user_state.pop(chat_id, None)
            self.show_admin_dashboard(message)
            return

        if current_state == 'admin_post_slip_waiting_text':
            # Store slip text and ask for photo
            self.user_state[chat_id]['data'] = {'text': message.text}
            self.user_state[chat_id]['state'] = 'admin_post_slip_waiting_photo'

            skip_keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True).add("⏩ Skip Photo", "❌ Cancel")

            bot.send_message(
                chat_id,
                "📸 **Step 2: Slip Photo**\n\n"
                "Please send the accompanying **PHOTO** for the slip, or tap '⏩ Skip Photo' if no photo is needed.",
                reply_markup=skip_keyboard,
                parse_mode='Markdown'
            )

        elif current_state == 'admin_post_slip_waiting_photo':
            # Handle photo or skip
            if message.content_type == 'photo':
                photo_id = message.photo[-1].file_id # Get the largest photo ID
                self.user_state[chat_id]['data']['photo_id'] = photo_id
                self.confirm_post_slip(message)
            elif message.text == "⏩ Skip Photo":
                self.user_state[chat_id]['data']['photo_id'] = "NO_PHOTO"
                self.confirm_post_slip(message)
            else:
                bot.send_message(chat_id, "❌ Invalid input. Please send a photo or tap '⏩ Skip Photo' or '❌ Cancel'.")

        elif current_state == 'admin_broadcast_waiting_message':
            # Finalize broadcast
            self.finalize_broadcast(message)

        else:
            # Not in an active admin flow
            pass

    @admin_only
    def confirm_post_slip(self, message):
        """Step 3: Admin confirms the final slip content."""
        chat_id = message.chat.id
        slip_data = self.user_state[chat_id]['data']

        confirm_keyboard = types.InlineKeyboardMarkup()
        confirm_keyboard.add(types.InlineKeyboardButton("✅ Confirm & Post Slip", callback_data="admin_post_slip_final"))
        confirm_keyboard.add(types.InlineKeyboardButton("❌ Cancel Post", callback_data="admin_cancel_post"))

        # Send a preview
        header = "✨ **PREVIEW: Daily Slip** ✨"
        self.send_slip(chat_id, slip_data, header)

        bot.send_message(chat_id, "👆 **Confirm Post?**\n\nThis will be sent to all active subscribers.", reply_markup=confirm_keyboard, parse_mode='Markdown')
        self.user_state[chat_id]['state'] = 'admin_post_slip_confirm' # Change state to wait for inline button

    @admin_only
    @log_action
    def finalize_post_slip(self, call):
        """Final step: Saves the slip to DB and broadcasts to subscribers."""
        chat_id = call.message.chat.id
        user_state_data = self.user_state.get(chat_id, {})

        if user_state_data.get('state') != 'admin_post_slip_confirm':
            bot.answer_callback_query(call.id, "❌ Action timed out or state is incorrect.")
            return

        slip_data = user_state_data.get('data')

        # 1. Save to DB
        slip_id = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        try:
            get_db_ref(f'slips/{slip_id}').set(slip_data)
            logger.info(f"New slip saved with ID: {slip_id}")

            # 2. Broadcast to all active subscribers
            self.broadcast_slip_to_subscribers(slip_data)

            bot.edit_message_text("✅ **Daily Slip Posted and Broadcasted!**", chat_id, call.message.message_id)
            self.user_state.pop(chat_id, None)
            self.show_admin_dashboard(call.message) # Return to dashboard

        except exceptions.FirebaseError as e:
            logger.error(f"Finalize post slip Firebase Error: {e}")
            bot.edit_message_text(f"❌ Error saving/broadcasting slip: {e}", chat_id, call.message.message_id)
            self.user_state.pop(chat_id, None)

    @admin_only
    def broadcast_slip_to_subscribers(self, slip_data):
        """Helper to send the slip to all currently active subscribers."""
        users = get_db_ref('users').get()
        if not users:
            logger.info("No users found for slip broadcast.")
            return

        header = "🎯 **New Verified Daily Slip is Here!**"

        count = 0
        for phone, user_data in users.items():
            try:
                # Check for active and not expired
                is_active = user_data.get('subscription_status') == 'active'
                is_expired = self.check_and_update_expiry(phone, user_data)

                if is_active and not is_expired:
                    self.send_slip(user_data['telegram_id'], slip_data, header)
                    count += 1
                    time.sleep(0.05) # Small delay for rate-limiting
            except Exception as e:
                logger.error(f"Error sending slip to user {phone}: {e}")

        logger.info(f"Successfully broadcasted slip to {count} active subscribers.")


    @admin_only
    def start_broadcast(self, message):
        """Step 1: Admin starts the broadcast process."""
        chat_id = message.chat.id

        keyboard = types.InlineKeyboardMarkup()
        keyboard.add(types.InlineKeyboardButton("All Users", callback_data="broadcast_filter_all"))
        keyboard.add(types.InlineKeyboardButton("Active Subscribers Only", callback_data="broadcast_filter_active"))
        keyboard.add(types.InlineKeyboardButton("❌ Cancel", callback_data="admin_cancel_post"))

        bot.send_message(
            chat_id,
            "📢 **Start Broadcast**\n\n"
            "Choose your target audience filter:",
            reply_markup=keyboard,
            parse_mode='Markdown'
        )

    @admin_only
    def handle_broadcast_filter(self, call):
        """Step 2: Admin chooses the broadcast filter."""
        chat_id = call.message.chat.id
        filter_type = call.data.split('_')[-1] # 'all' or 'active'

        self.user_state[chat_id] = {'state': 'admin_broadcast_waiting_message', 'filter': filter_type}

        cancel_keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True).add("❌ Cancel")

        bot.edit_message_text(
            f"🎯 **Target:** {'All Users' if filter_type == 'all' else 'Active Subscribers'}\n\n"
            "Please send the **MESSAGE** you wish to broadcast now. (Text only for now).",
            chat_id, call.message.message_id,
            reply_markup=cancel_keyboard,
            parse_mode='Markdown'
        )
        bot.answer_callback_query(call.id, "Filter selected.")

    @admin_only
    @log_action
    def finalize_broadcast(self, message):
        """Step 3: Sends the message to the filtered audience."""
        chat_id = message.chat.id
        current_state = self.user_state.get(chat_id, {})
        filter_type = current_state.get('filter')
        broadcast_text = message.text

        if not filter_type:
            bot.send_message(chat_id, "❌ Broadcast filter not set. Please restart the process.")
            return

        users = get_db_ref('users').get()
        if not users:
            bot.send_message(chat_id, "No users found in database.")
            self.user_state.pop(chat_id, None)
            return

        count = 0
        for phone, user_data in users.items():
            should_send = False
            user_tg_id = user_data.get('telegram_id')

            if filter_type == 'all':
                should_send = True
            elif filter_type == 'active':
                is_active = user_data.get('subscription_status') == 'active'
                is_expired = self.check_and_update_expiry(phone, user_data)
                if is_active and not is_expired:
                    should_send = True

            if should_send and user_tg_id:
                try:
                    bot.send_message(user_tg_id, f"📢 **AiBet Broadcast**\n\n{broadcast_text}", parse_mode='Markdown')
                    count += 1
                    time.sleep(0.05)
                except Exception as e:
                    logger.error(f"Failed to send broadcast to user {phone} ({user_tg_id}): {e}")

        bot.send_message(chat_id, f"✅ **Broadcast Complete!** Sent to **{count}** users (Filter: {filter_type}).")
        self.user_state.pop(chat_id, None)
        self.show_admin_dashboard(message)

    @admin_only
    @log_action
    def show_stats(self, message):
        """Displays key bot statistics."""
        chat_id = message.chat.id

        try:
            users = get_db_ref('users').get()

            total_users = 0
            active_subscriptions = 0
            total_referral_points = 0
            free_trial_claimed = 0

            if users:
                total_users = len(users)
                for phone, user_data in users.items():
                    # Check for active and not expired (and update status if needed)
                    is_active = user_data.get('subscription_status') == 'active'
                    is_expired = self.check_and_update_expiry(phone, user_data)

                    if is_active and not is_expired:
                        active_subscriptions += 1

                    total_referral_points += user_data.get('referral_points', 0)
                    if user_data.get('free_trial_claimed', False):
                        free_trial_claimed += 1

            # Simple uptime tracking (only works for the lifetime of the current process)
            uptime_seconds = time.time() - START_TIME
            uptime_str = str(datetime.timedelta(seconds=int(uptime_seconds)))


            stats_msg = (
                "📈 **AiBet Bot Statistics**\n\n"
                f"**Total Registered Users:** **{total_users}**\n"
                f"**Active Subscriptions:** **{active_subscriptions}**\n"
                f"**Free Trials Used:** **{free_trial_claimed}**\n\n"
                f"**Referral Summary:**\n"
                f"  - Total Points Earned: **{total_referral_points}**\n"
                f"  - Equivalent Free Days: **{total_referral_points // REFERRAL_POINTS_FOR_DAY}**\n\n"
                f"**Technical Info:**\n"
                f"  - Bot Version: 1.0.0\n"
                f"  - Uptime: {uptime_str}\n"
            )

            bot.send_message(chat_id, stats_msg, parse_mode='Markdown')

        except exceptions.FirebaseError as e:
            logger.error(f"Error fetching stats: {e}")
            bot.send_message(chat_id, "❌ Error retrieving bot statistics.")

    @admin_only
    @log_action
    def show_user_list(self, message):
        """Displays a summary of the user list."""
        chat_id = message.chat.id

        try:
            users = get_db_ref('users').get()

            if not users:
                bot.send_message(chat_id, "No users registered yet.")
                return

            user_summary = []
            for phone, user_data in users.items():
                status = user_data.get('subscription_status', 'inactive')
                # Only show active users in the summary list
                if status == 'active':
                    name = user_data.get('full_name', 'N/A')
                    expiry = user_data.get('subscription_expiry_date', 'N/A')
                    user_summary.append(f"• {name} ({phone}) - Expires: {expiry[:10]}")

            summary_msg = (
                "👤 **User List Summary**\n\n"
                f"**Total Users:** {len(users)}\n"
                f"**Active Subscribers ({len(user_summary)}):**\n"
                f"{'\n'.join(user_summary[:10])}\n"
                f"... and {'more' if len(user_summary) > 10 else 'that\'s all'} (Full list too long for Telegram message)\n\n"
                "Use the full Firebase console for detailed user management."
            )
            bot.send_message(chat_id, summary_msg, parse_mode='Markdown')

    @admin_only
    @log_action
    def run_db_cleanup(self, message):
        """Utility to clean up old, inactive users (older than 60 days)."""
        chat_id = message.chat.id
        DAYS_INACTIVE_FOR_CLEANUP = 60

        try:
            users = get_db_ref('users').get()

            if not users:
                bot.send_message(chat_id, "Database is empty, no cleanup required.")
                return

            cleanup_count = 0
            now = datetime.datetime.now()

            for phone, user_data in users.items():
                status = user_data.get('subscription_status', 'inactive')
                last_action_str = user_data.get('last_action') # Tracked in registration/payment
                created_at_str = user_data.get('created_at')

                if status == 'inactive' and (last_action_str or created_at_str):
                    try:
                        # Use the latest of last_action or created_at
                        check_dt_str = last_action_str if last_action_str else created_at_str
                        check_dt = datetime.datetime.fromisoformat(check_dt_str)

                        if (now - check_dt).days > DAYS_INACTIVE_FOR_CLEANUP:
                            # User is inactive and older than the threshold, proceed to delete
                            get_db_ref(f'users/{phone}').delete()
                            cleanup_count += 1
                            logger.info(f"Cleaned up inactive user: {phone}")
                    except ValueError:
                        logger.error(f"Could not parse date for user {phone}")

            bot.send_message(chat_id, f"🧹 **Database Cleanup Complete!** Removed **{cleanup_count}** inactive users older than {DAYS_INACTIVE_FOR_CLEANUP} days.")

        except exceptions.FirebaseError as e:
            logger.error(f"Error during DB cleanup: {e}")
            bot.send_message(chat_id, "❌ Error during database cleanup.")


# --- BOT HANDLER INSTANTIATION ---
manager = AiBetBotManager()
START_TIME = time.time()

# --- TELEGRAM HANDLERS (Mapping commands to Manager methods) ---

# Command Handlers
@bot.message_handler(commands=['start', 'help', 'admin', 'stats', 'postslip', 'broadcast', 'dbcleanup'])
@rate_limit
def handle_commands(message):
    command = message.text.split()[0].lower()

    if command == '/start':
        manager.start_command(message)
    elif command == '/help':
        manager.show_help(message)
    elif command == '/admin':
        manager.show_admin_dashboard(message)
    # Admin commands called directly
    elif command == '/stats':
        manager.show_stats(message)
    elif command == '/postslip':
        manager.start_post_slip(message)
    elif command == '/broadcast':
        manager.start_broadcast(message) # Starts with filter selection inline keyboard
    elif command == '/dbcleanup':
        manager.run_db_cleanup(message)

# Content Handlers
@bot.message_handler(content_types=['contact'])
@rate_limit
def handle_contact_message(message):
    manager.handle_contact(message)

@bot.message_handler(content_types=['text', 'photo'])
@rate_limit
def handle_general_message(message):
    chat_id = message.chat.id
    current_state = manager.user_state.get(chat_id, {}).get('state')

    # Prioritize multi-step admin input
    if current_state and current_state.startswith('admin_'):
        manager.handle_admin_input(message)
        return

    # User Dashboard button clicks (text messages that are not part of an active flow)
    manager.handle_dashboard_button(message)


# Inline Button Handlers
@bot.callback_query_handler(func=lambda call: call.data.startswith('plan_') or call.data == 'show_plans')
@rate_limit
def handle_plan_callback(call):
    if call.data.startswith('plan_'):
        manager.process_plan_selection(call)
    elif call.data == 'show_plans':
        # Simulate button press to show plans
        manager.handle_subscribe(call.message)
        bot.answer_callback_query(call.id) # Dismiss alert

@bot.callback_query_handler(func=lambda call: call.data == 'redeem_points')
@rate_limit
def handle_redeem_callback(call):
    manager.redeem_points(call)

@bot.callback_query_handler(func=lambda call: call.data.startswith('admin_post_slip_') or call.data.startswith('admin_broadcast_') or call.data == 'admin_cancel_post')
@admin_only
def handle_admin_callback(call):
    chat_id = call.message.chat.id
    if call.data == 'admin_post_slip_final':
        manager.finalize_post_slip(call)
    elif call.data.startswith('admin_broadcast_filter_'):
        manager.handle_broadcast_filter(call)
    elif call.data == 'admin_cancel_post':
        manager.user_state.pop(chat_id, None)
        bot.edit_message_text("❌ Action cancelled.", chat_id, call.message.message_id)
        bot.answer_callback_query(call.id)
        manager.show_admin_dashboard(call.message)


# Payment Handlers
@bot.pre_checkout_query_handler(func=lambda query: True)
def handle_pre_checkout(pre_checkout_query):
    manager.handle_pre_checkout_query(pre_checkout_query)

@bot.message_handler(content_types=['successful_payment'])
def handle_payment_success(message):
    manager.handle_successful_payment(message)

# --- FLASK & WEBHOOK SETUP FOR RENDER ---

@app.route(WEBHOOK_PATH, methods=['POST'])
def webhook():
    if request.headers.get('content-type') == 'application/json':
        json_string = request.get_data().decode('utf-8')
        update = telebot.types.Update.de_json(json_string)
        bot.process_new_updates([update])
        return '!', 200
    else:
        abort(403) # Bad request

@app.route('/')
def index():
    # Simple check for the health of the application
    return "AiBet Bot is running. Listening for webhooks.", 200

def set_webhook():
    """Sets the Telegram webhook URL."""
    try:
        # Remove old webhook before setting a new one
        bot.remove_webhook()
        time.sleep(0.5)

        # Set the new webhook
        success = bot.set_webhook(url=WEBHOOK_URL)
        if success:
            logger.info(f"Webhook successfully set to: {WEBHOOK_URL}")
        else:
            logger.error("Failed to set webhook.")
        return success
    except Exception as e:
        logger.error(f"Error setting webhook: {e}")
        return False


# --- DEPLOYMENT INSTRUCTIONS ---
"""
## 🚀 Render Deployment Guide for AiBet Bot

This bot is configured to run on Render using a **Webhook** and a **Flask** server.

1.  **Dependencies (`requirements.txt`):**

pytelegrambotapi
    Flask
    firebase-admin
    gunicorn  # Recommended for production hosting
    ```

2.  **Configuration:**
    * **Edit `aibet_bot.py`:** Replace all placeholder values in the `CONFIGURATION & SETUP` section:
        * `TOKEN`: Your Telegram Bot Token.
        * `WEBHOOK_URL_BASE`: The external URL of your Render Service (e.g., `https://aibet-bot.onrender.com`).
        * `FIREBASE_DATABASE_URL`: Your Firebase Realtime Database URL.
        * `ADMIN_IDS`: Your Telegram User IDs (as integers).
        * `CHAPA_PROVIDER_TOKEN`: Your Telegram Chapa Payment Provider Token.
    * **Firebase Credentials (IMPORTANT):** For a secure, production-ready setup, **DO NOT hardcode** your Firebase Service Account JSON.
        * On Render, you should load the JSON content from an Environment Variable (e.g., `FIREBASE_CREDENTIALS_JSON`) and configure `firebase_admin` to use it. The current setup is simplified for Pydroid/single-file demonstration and may need adjustment for secure Render deployment.

3.  **Render Service Setup:**
    * **Type:** Web Service
    * **Build Command:** `pip install -r requirements.txt`
    * **Start Command:** `gunicorn aibet_bot:app` (This tells Gunicorn to run the Flask app object named `app` in the `aibet_bot.py` file).
    * **Port:** Ensure your app listens on the port provided by the environment, which is handled by `LISTEN_PORT = int(os.environ.get('PORT', 5000))` in the code.

4.  **Initial Webhook Setting (Crucial):**
    * Since the webhook must be set before Telegram sends updates, you need to run `set_webhook()` once.
    * **Recommended Method:** Run the Python script *locally* just to execute `set_webhook()` after you get the external Render URL.

In [ ]:
# Local execution for webhook setting
        if __name__ == '__main__':
            # Only run this locally ONCE after Render is deployed and URL is known.
            # Comment out after successful execution.
            print("Running webhook setup...")
            if set_webhook():
                print("Webhook set successfully. Deploy to Render now.")
            else:
                print("Webhook failed to set.")

* Alternatively, you can manually call the `set_webhook` endpoint via your browser or a tool like Postman after your Render service is live: `https://api.telegram.org/bot<TOKEN>/setWebhook?url=<WEBHOOK_URL>`.

5.  **Run:** Once the webhook is set and the service is deployed, the bot will start receiving updates from Telegram via the `/webhook` endpoint.
"""

if __name__ == '__main__':
    # This block should primarily be used for local testing (Pydroid 3)
    # or for the ONE-TIME webhook setting.
    
    # In a production Render deployment, Gunicorn/Flask manages the process.
    # The set_webhook should be done externally or at a defined startup hook.
    
    # --- Local Test/Webhook Setup (Modify as needed) ---
    if set_webhook():
        # Only run Flask's development server if running locally for testing.
        # Use gunicorn in production.
        logger.info("Starting Flask server for local testing...")
        # Note: In Pydroid 3, you might need to use app.run(host='0.0.0.0', port=5000)
        # However, for a proper Render setup, the `gunicorn aibet_bot:app` command handles this.
        # For simplicity and to allow a local test run:
        # app.run(host='0.0.0.0', port=LISTEN_PORT)
        
        # We will not run app.run() here to avoid interfering with the gunicorn
        # start command on Render. This file's primary purpose is to be run by gunicorn.
        pass
    else:
        logger.error("Initial setup failed. Check configuration and network.")
````

I seem to be encountering an error. Can I try something else for you?